Import the necessary libraries:


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import DataLoader

Load and preprocess the CIFAR10 dataset:


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)




test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)



Subset 0,1

In [ ]:
from torch.utils.data import Subset
def create_subset(dataset, labels):
    indices = [idx for idx, label in enumerate(dataset.targets) if label in labels]
    subset = Subset(dataset, indices)
    return subset


In [ ]:
# Specify the labels for the subset
subset_labels = [0, 1]
# Create the subset
subset = create_subset(train_dataset, subset_labels)
train_loader_2 = DataLoader(subset, batch_size=25, shuffle=True)


In [ ]:
# Specify the labels for the subset
subset_labels = [0, 1,2]
# Create the subset
subset = create_subset(train_dataset, subset_labels)

train_loader_3 = DataLoader(subset, batch_size=25, shuffle=True)


In [ ]:
# Specify the labels for the subset
subset_labels = [0, 1,2,3]
# Create the subset
subset = create_subset(train_dataset, subset_labels)

train_loader_4 = DataLoader(subset, batch_size=25, shuffle=True)


In [ ]:
# Specify the labels for the subset
subset_labels = [0, 1,2,3,4]
# Create the subset
subset = create_subset(train_dataset, subset_labels)

train_loader_5 = DataLoader(subset, batch_size=25, shuffle=True)


In [ ]:
# Specify the labels for the subset
subset_labels = [0, 1,2,3,4,5]
# Create the subset
subset = create_subset(train_dataset, subset_labels)

train_loader_6 = DataLoader(subset, batch_size=25, shuffle=True)


In [ ]:
# Specify the labels for the subset
subset_labels = [0, 1,2,3,4,5,6]
# Create the subset
subset = create_subset(train_dataset, subset_labels)

train_loader_7 = DataLoader(subset, batch_size=25, shuffle=True)


In [ ]:
# Specify the labels for the subset
subset_labels = [0, 1,2,3,4,5,6,7]
# Create the subset
subset = create_subset(train_dataset, subset_labels)

train_loader_8 = DataLoader(subset, batch_size=25, shuffle=True)


In [ ]:
# Specify the labels for the subset
subset_labels = [0, 1,2,3,4,5,6,7,8]
# Create the subset
subset = create_subset(train_dataset, subset_labels)

train_loader_9= DataLoader(subset, batch_size=25, shuffle=True)

In [ ]:
test_dataset

Load the pre-trained WideResNet model:


In [ ]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")  # Set the device to GPU
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")  # Set the device to CPU
    print("Using CPU")

In [ ]:

model = torchvision.models.wide_resnet50_2(pretrained=True)
model = model.to(device)
model.eval()


Modify the model by removing the output layer and keeping the features:


In [ ]:
model = nn.Sequential(*list(model.children())[:-1])


Create a new fully connected network for classification:


In [ ]:
# Define the network class
class SimpleNetwork(nn.Module):
    def __init__(self, num_outputs):
        super(SimpleNetwork, self).__init__()

        self.input_layer = nn.Linear(2048, 64)
        self.hidden_layer = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.output_layer = nn.Linear(32, num_outputs)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        x = x.to(self.input_layer.weight.device)  # Move input tensor to the same device as the parameters
        x1 = self.input_layer(x)
        x1_relu  = self.relu(x1)
        x2  = self.hidden_layer(x1_relu)
        x2_relu  = self.relu(x2)
        x = self.output_layer(x2_relu)
        x = self.sigmoid(x)
        return x,x2_relu,x1_relu



# Specify the number of outputs for the network
# num_outputs = 10  # You can change this number based on your needs

# # Create an instance of the network
# network = SimpleNetwork(num_outputs)

In this updated code, I modified the forward method of the NuSALoss class to include an additional input parameter inputs, which represents the input sample to each layer (xl).

Within the loop that iterates over the named parameters of the model, xl is now set as the inputs passed to the forward method. The NuSA term is computed by performing the necessary matrix multiplications and norms between column_space_basis and xl.

By including inputs as an argument and using it in the NuSA term calculation, the code now incorporates the input sample for each layer into the loss function, as specified in the equation you provided.

Once again, I apologize for the confusion caused by the incorrect information in my previous response.

In [ ]:
import torch
import torch.nn as nn

# Define the loss function with NuSA term
class NuSALoss(nn.Module):
    def __init__(self, lambda_value):
        super(NuSALoss, self).__init__()
        self.lambda_value = lambda_value

    def forward(self, outputs, targets, model, inputs):
        standard_loss = nn.CrossEntropyLoss()(outputs, targets)
        nusa_term = 0.0

        i = 0
        for name, param in model.named_parameters():
            if 'weight' in name:
                weight = param.data
                xl = inputs[i]  # Input sample to the layer
                xl = xl.to(device)


                i += 1
                Q, _  = torch.qr(weight.t())
                column_space_basis = Q.t()
                column_space_basis = column_space_basis.to(device)
                nusa_term += torch.norm(torch.matmul(torch.matmul(column_space_basis.t(),
                                                                  column_space_basis), xl.t())) / xl.norm()

        loss = standard_loss + self.lambda_value * nusa_term
        return loss

# Specify the lambda value for the tradeoff between the standard loss and the NuSA term
lambda_value = 0.1  # You can change this value based on your needs

# Create an instance of the loss function
loss_function = NuSALoss(lambda_value)


In [ ]:
# criterion = nn.CrossEntropyLoss()
def train(train_loader,network):

    optimizer = torch.optim.Adam(network.parameters(), lr=0.01)
    global loss_function
    # new_train_loader = make_train_loader_from_number_class_we_want(number_of_known_classes)
    for epoch in range(20):
        print(epoch)
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass through the WideResNet model
            with torch.no_grad():
                features = model(images)

            # Flatten the features
            features = features.view(features.size(0), -1)
            # Forward pass through the classifier
            outputs,relue_x2,relue_x1 = network(features)
            inputs = [features,relue_x1,relue_x2]
            # Move input and target tensors to the same device
            outputs = outputs.to(device)
            loss = loss_function(outputs, labels,network,inputs)
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'loss: {loss}')

    return network

Evaluate the classifier's performance:
`

In [ ]:
def nusa_testing(data_loader, classifier, threshold):
    outlier_indicator = []
    outlier_class_labels = []
    global model
    classifier.eval()  # Set the model to evaluation mode

    with torch.no_grad():
        for inputs, labels in data_loader:
            # print(len(inputs))
            inputs = inputs.to(device)
            labels = labels.to(device)

            features = model(inputs)

            # Flatten the features
            features = features.view(features.size(0), -1)


            outputs,relue_x2,relu_x1 = classifier(features)  # Compute forward pass to get output for the sample
            nusa = compute_nusa(classifier,[features,relu_x1,relue_x2])  # Compute NuSA for the sample

            if nusa > threshold:
                _, predicted = torch.max(outputs, 1)
                outlier_indicator.append(False)
                outlier_class_labels.append(predicted.item())
            else:
                outlier_indicator.append(True)
                outlier_class_labels.append(None)

    return outlier_indicator, outlier_class_labels

def compute_nusa(model,inputs):
    nusa = 0.0
    i = 0
    for name, param in model.named_parameters():
        if 'weight' in name:
            weight = param.data
            xl = inputs[i]
            xl = xl.to(device)
            i += 1
            # Reshape the weight tensor
            Q,_ = torch.qr(weight.t())
            column_space_basis = Q.t()
            column_space_basis = column_space_basis.to(device)
            nusa += torch.norm(torch.matmul(torch.matmul(column_space_basis.t(),
                                                         column_space_basis), xl.t())) / xl.norm()

    return nusa

In [ ]:


def test(test_loader,network,list_of_acceptable_label,thershold=0.1):

    outlier_indicator, outlier_class_labels = nusa_testing(test_loader,network,thershold)

    correct = 0
    total = 0
    for outlier_indi,outlier_labels,test in zip(outlier_indicator, outlier_class_labels,test_loader):

        if test[1] in list_of_acceptable_label:
          total += 1

        if not outlier_indi:
            if outlier_labels == test[1]:
              correct += 1



    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy}%")
    return accuracy

In [ ]:
import joblib
from google.colab import drive
drive.mount('/content/drive')
# test_res = []

file_path = "/content/drive/MyDrive/data_maining_project/test_res_20.pkl"

# Write the content to the file
# joblib.dump(test_res, file_path)

# test_res = []

In [ ]:

# Create an instance of the network for known 2
network = SimpleNetwork(2)
network = train(train_loader_2,network)


In [ ]:
res = test(test_loader,network,[0,1],0.1)
test_res.append(res)
print(res)


In [ ]:
test_res = []

In [ ]:
# Write the content to the file
joblib.dump(test_res, file_path)


In [ ]:

# Create an instance of the network for known 3
network = SimpleNetwork(3)
network = train(train_loader_3,network)



In [ ]:
res = test(test_loader,network,[0,1,2],0.3)
test_res.append(res)
print(res)

In [ ]:
test_res = [test_res[0] ,test_res[-1]]
test_res

In [ ]:
# Write the content to the file
joblib.dump(test_res, file_path)


In [ ]:
# Create an instance of the network for known 4
network = SimpleNetwork(4)
network = train(train_loader_4,network)


In [ ]:
res = test(test_loader,network,[0,1,2,3],0.4)
test_res.append(res)
print(res)


In [ ]:
test_res = [test_res[0],test_res[1],test_res[-1]]
test_res

In [ ]:
# Write the content to the file
joblib.dump(test_res, file_path)


In [ ]:
# Create an instance of the network for known 5
network = SimpleNetwork(5)
network = train(train_loader_5,network)


In [ ]:
res = test(test_loader,network,[0,1,2,3,4],0.1)
test_res.append(res)
print(res)


In [ ]:
test_res = [test_res[0],test_res[1],test_res[2],test_res[-1]]

In [ ]:
test_res = joblib.load("/content/drive/MyDrive/data_maining_project/test_res_20.pkl")

In [ ]:
# Write the content to the file
joblib.dump(test_res, file_path)


In [ ]:
test_res

In [ ]:
# Create an instance of the network for known 6
network = SimpleNetwork(6)
network = train(train_loader_6,network)



In [ ]:
res = test(test_loader,network,[0,1,2,3,4,5],0)
test_res.append(res)
print(res)

In [ ]:
test_res

In [ ]:
test_res = [test_res[0],test_res[1],test_res[2],test_res[3],test_res[4],test_res[5]]

In [ ]:
# Write the content to the file
joblib.dump(test_res, file_path)


In [ ]:
# Create an instance of the network for known 7
network = SimpleNetwork(7)
network = train(train_loader_7,network)



In [ ]:
res = test(test_loader,network,[0,1,2,3,4,5,6],0.01)
print(res)

In [ ]:
test_res.append(res)
# Write the content to the file
joblib.dump(test_res, file_path)


In [ ]:
# Create an instance of the network for known 8
network = SimpleNetwork(8)
network = train(train_loader_8,network)


In [ ]:
res = test(test_loader,network,[0,1,2,3,4,5,6,7],0.3)

print(res)

In [ ]:
test_res.append(res)


In [ ]:
# Write the content to the file
joblib.dump(test_res, file_path)


In [ ]:
# Create an instance of the network for known 9
network = SimpleNetwork(9)
network = train(train_loader_9,network)


In [ ]:
res = test(test_loader,network,[0,1,2,3,4,5,6,7,8],1.2)

print(res)


In [ ]:
test_res.append(res)


In [ ]:
# Write the content to the file
joblib.dump(test_res, file_path)


In [ ]:
import joblib
from google.colab import drive
drive.mount('/content/drive')

file_path = "/content/drive/MyDrive/data_maining_project/test_res_20.pkl"

# Write the content to the file
loaded_object = joblib.load(file_path)




In [ ]:
loaded_object

In [ ]:
import matplotlib.pyplot as plt

# List of labels for the x-axis
labels = [i for i in range(2,10)]

# Create the bar graph
plt.bar(labels, loaded_object)

# Add labels and title
plt.xlabel('Categories')
plt.ylabel('Values')
plt.title('Bar Graph')

# Display the graph
plt.show()

In [ ]:
import joblib

res_nsu = joblib.load('test_res_nsu.pkl')
res_without_nsu = joblib.load('test_res_withouth_nsu.pkl')



import matplotlib.pyplot as plt
import numpy as np

# List of labels for the x-axis
labels = [i for i in range(2,10)]

# Set the width of each bar
bar_width = 0.35

# Create the x-axis values
x = np.arange(len(labels))

# Create the grouped bar graph
fig, ax = plt.subplots()
rects1 = ax.bar(x - bar_width/2, res_nsu, bar_width, label='NuSA')
rects2 = ax.bar(x + bar_width/2, res_without_nsu, bar_width, label='No NuSA')

# Add labels and title
ax.set_xlabel('Categories')
ax.set_ylabel('Values')
ax.set_title('Comparison two NuSA and No NuSA')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# Display the graph
plt.show()

In [ ]:
import numpy as np


def nusa_testing_for_precission_recall(data_loader, classifier, threshold):
    outlier_indicator = []
    outlier_class_labels = []
    global model
    classifier.eval()  # Set the model to evaluation mode

    with torch.no_grad():
        for inputs, labels in data_loader:
            # print(len(inputs))
            inputs = inputs.to(device)
            labels = labels.to(device)

            features = model(inputs)

            # Flatten the features
            features = features.view(features.size(0), -1)


            outputs,relue_x2,relu_x1 = classifier(features)  # Compute forward pass to get output for the sample
            nusa = compute_nusa(classifier,[features,relu_x1,relue_x2])  # Compute NuSA for the sample
            print(nusa)
            outputs = outputs[0].numpy()
            outputs = outputs.tolist()
            outputs.append(0)
            # print(outputs)
            if nusa > threshold:
                outlier_indicator.append(False)
                outlier_class_labels.append(outputs)
            else:
                outlier_indicator.append(True)
                outlier_class_labels.append(np.array([0,0,0,0,0,1]))

    return outlier_indicator, outlier_class_labels

def compute_nusa(model,inputs):
    nusa = 0.0
    i = 0
    for name, param in model.named_parameters():
        if 'weight' in name:
            weight = param.data
            xl = inputs[i]
            xl = xl.to(device)
            i += 1
            # Reshape the weight tensor
            # weight_reshaped = weight.permute(1, 0, 2, 3).reshape(weight.size(1), -1)

            # Q, _  = torch.qr(weight_reshaped.t())
            Q,_ = torch.qr(weight.t())
            column_space_basis = Q.t()

            # column_space_basis = column_space_basis.to(device)
            # nusa += torch.norm(torch.matmul(torch.matmul(column_space_basis.t(), column_space_basis), x.t())) / x.norm()

            column_space_basis = column_space_basis.to(device)
            nusa += torch.norm(torch.matmul(torch.matmul(column_space_basis.t(), column_space_basis), xl.t())) / xl.norm()

    return nusa

In [ ]:
def test_for_precission_recall(test_loader,network,list_of_acceptable_label):

    outlier_indicator, outlier_class_labels = nusa_testing_for_precission_recall(test_loader,network,1.8)
    print(outlier_indicator)

    true_label = []
    predict_label = []

    for outlier_indi,outlier_labels,test in zip(outlier_indicator, outlier_class_labels,test_loader):

        if test[1] in list_of_acceptable_label :
          true_label.append(test[1])
        else :
          true_label.append(5)

        if not outlier_indi:
            predict_label.append(outlier_labels)
        else:
             predict_label.append(outlier_labels)


    return true_label,predict_label


In [ ]:
# Create an instance of the network for known 5
# network = SimpleNetwork(5)
# network = train(train_loader_5,network)
true_labels,predict_labels = test_for_precission_recall(test_loader,network,[0,1,2,3,4])



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score

# Assuming you have the true labels and predicted probabilities for each class
true_labels = np.array(true_labels)  # True labels for the samples
predicted_probs = np.array(predict_labels)  # Predicted probabilities for each class (shape: 5x5)
# print(predict_labels)
# Plot Precispredict_labelsion-Recall curves for each class
plt.figure()

for class_label in range(6):
    binary_true_labels = [1 if label == class_label else 0 for label in true_labels]
    prob_scores = predicted_probs[:, class_label]
    precision, recall, _ = precision_recall_curve(binary_true_labels, prob_scores)
    average_precision = average_precision_score(binary_true_labels, prob_scores)
    plt.plot(recall, precision, label=f'Class {class_label} (AP = {average_precision:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for (Five known Classes)')
plt.legend(loc='lower left')
plt.grid(True)
plt.show()
